In [13]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,ExtraTreesClassifier 
from sklearn import preprocessing
import pickle

In [2]:
names = ["XGBoost"]

In [3]:
classifiers = [xgb.XGBClassifier(learning_rate=0.025, max_depth=5, n_estimators=700,early_stopping_rounds=10)]

In [4]:
df = pd.read_csv('quora_features.csv')
df.shape

(404290, 34)

In [5]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [6]:
Z= df.drop(['question1','question2'], 1)
Z.head()

,id,qid1,qid2,is_duplicate,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,...,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,0,1,2,0,66,57,9,20,20,14,...,5.081614,1.0,94.023324,0.371408,0.168999,0.186557,0.031817,-0.091902,0.050416,0.337301
1,1,3,4,0,51,88,-37,21,29,8,...,14.195119,1.0,177.588090,1.012091,0.455910,0.592655,0.008735,0.094704,0.284010,-0.034444
2,2,5,6,0,73,59,14,25,24,14,...,9.055989,1.0,135.988707,0.666346,0.307828,0.342306,0.239752,0.144554,0.026759,-0.474131
3,3,7,8,0,50,65,-15,19,26,11,...,15.987437,1.0,192.237828,1.140536,0.506028,0.692421,-0.002527,0.069649,-0.244560,-0.265568
4,4,9,10,0,76,39,37,25,18,13,...,12.103178,1.0,161.408435,0.860225,0.382770,0.480633,-0.133849,0.114777,0.217900,-0.338876


In [7]:
Z=clean_dataset(Z)
y = Z['is_duplicate']
X = Z.drop(['is_duplicate'], 1)
X

,id,qid1,qid2,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,...,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,0.0,1.0,2.0,66.0,57.0,9.0,20.0,20.0,14.0,12.0,...,5.081614,1.0,94.023324,0.371408,0.168999,0.186557,0.031817,-0.091902,0.050416,0.337301
1,1.0,3.0,4.0,51.0,88.0,-37.0,21.0,29.0,8.0,13.0,...,14.195119,1.0,177.588090,1.012091,0.455910,0.592655,0.008735,0.094704,0.284010,-0.034444
2,2.0,5.0,6.0,73.0,59.0,14.0,25.0,24.0,14.0,10.0,...,9.055989,1.0,135.988707,0.666346,0.307828,0.342306,0.239752,0.144554,0.026759,-0.474131
3,3.0,7.0,8.0,50.0,65.0,-15.0,19.0,26.0,11.0,9.0,...,15.987437,1.0,192.237828,1.140536,0.506028,0.692421,-0.002527,0.069649,-0.244560,-0.265568
4,4.0,9.0,10.0,76.0,39.0,37.0,25.0,18.0,13.0,7.0,...,12.103178,1.0,161.408435,0.860225,0.382770,0.480633,-0.133849,0.114777,0.217900,-0.338876
5,5.0,11.0,12.0,86.0,90.0,-4.0,26.0,27.0,16.0,16.0,...,10.254048,1.0,150.335589,0.741946,0.337470,0.400061,0.102946,-0.077741,-0.335505,-0.057847
6,6.0,13.0,14.0,19.0,62.0,-43.0,14.0,20.0,4.0,11.0,...,17.414754,1.0,200.598725,1.281122,0.597082,0.816676,-0.090366,-0.000266,0.296892,0.130717
7,7.0,15.0,16.0,30.0,41.0,-11.0,16.0,16.0,7.0,9.0,...,5.008762,1.0,95.395891,0.359486,0.161125,0.187150,0.346824,0.368469,0.635773,0.865446
8,8.0,17.0,18.0,31.0,37.0,-6.0,16.0,16.0,8.0,8.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.202336,0.202336,-0.434927,-0.434927
9,9.0,19.0,20.0,60.0,49.0,11.0,25.0,24.0,9.0,9.0,...,10.634987,1.0,153.647611,0.754599,0.337939,0.409126,0.120050,0.204449,-0.214573,-0.126814


In [8]:
X=X.drop(['id','qid1','qid2'],1)

In [9]:
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, stratify=Z.is_duplicate,random_state=42)
feat_test=DecisionTreeClassifier(min_samples_split=0.1)
feat_test.fit(X_train, y_train)
print("\n\nImportant Features:\n",feat_test.feature_importances_)



Important Features:
 [2.54478432e-06 8.11367552e-06 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.24243742e-03 3.95730384e-02
 1.25480016e-01 0.00000000e+00 1.90677110e-02 0.00000000e+00
 0.00000000e+00 6.66821853e-02 2.71804059e-02 5.91805617e-03
 6.99935804e-01 1.49096873e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]


In [18]:
# iterate over classifiers
for name, clf in zip(names, classifiers):
        clf.fit(X, y)
        score = clf.score(X_test, y_test)
        print(name,'Score:',score)
        prediction1 = clf.predict(X_test)
        np.set_printoptions(precision=2)

XGBoost Score: 0.7585420253845092


In [19]:
clf.score(X_train,y_train)

0.757778116821388

In [20]:
model_pkl = open('model.pkl', 'wb')
pickle.dump(clf, model_pkl)

In [21]:
# Loading the saved decision tree model pickle
loaded_pkl= open('model.pkl', 'rb')
loaded_model = pickle.load(loaded_pkl)

In [23]:
loaded_model.score(X_test,y_test)

0.7585420253845092

In [24]:
scalerfile = 'scaler.sav'
pickle.dump(min_max_scaler, open(scalerfile, 'wb'))